In [ ]:
import os
from google.colab import userdata

groq_api_key = userdata.get('groq_api_key')
os.environ["GROQ_API_KEY"] = groq_api_key

In [ ]:
!pip install groq
!pip install gradio

  Using cached gradio-5.29.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.10.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached ruff-0.11.8-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached starlette-0.46.2-py3-none-any.whl.metadata (6.2 kB)
  Using cached tomlkit-0.13.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached uvicorn-0.34.2-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import json
import re
from typing import Dict, List, Any
import gradio as gr

# Install required packages if not already installed
# Uncomment these lines if you need to install packages
# import subprocess
# subprocess.run(["pip", "install", "-q", "groq", "gradio"])

from groq import Groq

class CollegeCounselorChatbot:
    def __init__(self, api_key, name="Lauren"):
        self.name = name
        self.model = "llama-3.1-8b-instant"

        # Initialize Groq client with the provided API key
        self.client = Groq(api_key=api_key)

        self.conversation_history = [
            {"role": "system", "content": f"""
            You are {name}, an AI college counselor for Indian students. Your goal is to collect information about the student and provide personalized college recommendations.

            You need to gather these key details from the student through natural conversation:
            1. Marks_10th - Student's 10th standard marks percentage
            2. Marks_12th - Student's 12th standard marks percentage
            3. JEE_Score - JEE score if applicable
            4. Budget - How much they can afford for their entire education
            5. Preferred_Location - Which part of India they prefer to study in
            6. Gender - Student's gender
            7. Target_Exam - Which entrance exams they're targeting
            8. State_Board - Which educational board they studied under
            9. Category - Their reservation category (General, OBC, SC, ST, etc.)
            10. Extra_Curriculars - Any extracurricular activities/achievements
            11. Future_Goal - Career aspirations or goals

            Be friendly, conversational, and encouraging. First introduce yourself briefly and start collecting information.
            Only move on to college recommendations after collecting all the necessary information.

            When making recommendations, suggest 3-5 colleges that match their profile, explaining why each college is suitable,
            key programs offered, admission requirements, and costs. Also provide practical next steps for application.
            """}
        ]
        self.student_data = {
            'Marks_10th': None,
            'Marks_12th': None,
            'JEE_Score': None,
            'Budget': None,
            'Preferred_Location': None,
            'Gender': None,
            'Target_Exam': None,
            'State_Board': None,
            'Category': None,
            'Extra_Curriculars': None,
            'Future_Goal': None
        }
        self.data_collected = False

        # Sample college database
        self.colleges = [
            {"name": "IIT Bombay", "min_jee": 8000, "fees": 800000, "location": "Mumbai", "acceptance_rate": "Very Low", "specialties": ["Engineering", "Technology"]},
            {"name": "IIT Delhi", "min_jee": 9000, "fees": 750000, "location": "Delhi", "acceptance_rate": "Very Low", "specialties": ["Engineering", "Computer Science"]},
            {"name": "BITS Pilani", "min_jee": 15000, "fees": 1200000, "location": "Rajasthan", "acceptance_rate": "Low", "specialties": ["Engineering", "Pharmacy"]},
            {"name": "VIT Vellore", "min_jee": 50000, "fees": 900000, "location": "Tamil Nadu", "acceptance_rate": "Moderate", "specialties": ["Engineering", "Bio-Technology"]},
            {"name": "Manipal Institute of Technology", "min_jee": 70000, "fees": 1500000, "location": "Karnataka", "acceptance_rate": "Moderate", "specialties": ["Engineering", "Medicine"]},
            {"name": "NIT Trichy", "min_jee": 20000, "fees": 500000, "location": "Tamil Nadu", "acceptance_rate": "Low", "specialties": ["Engineering"]},
            {"name": "Delhi University", "min_jee": None, "fees": 200000, "location": "Delhi", "acceptance_rate": "Moderate", "specialties": ["Arts", "Commerce", "Science"]},
            {"name": "AIIMS Delhi", "min_jee": None, "fees": 600000, "location": "Delhi", "acceptance_rate": "Very Low", "specialties": ["Medicine"]},
            {"name": "St. Stephen's College", "min_jee": None, "fees": 300000, "location": "Delhi", "acceptance_rate": "Low", "specialties": ["Arts", "Science"]},
            {"name": "Christ University", "min_jee": None, "fees": 500000, "location": "Bangalore", "acceptance_rate": "Moderate", "specialties": ["Commerce", "Management"]},
        ]

    def extract_information(self, message):
        """Extract student information from user messages with improved patterns"""
        # Define patterns to recognize information (improved for better recognition)
        patterns = {
            'Marks_10th': r'(?:10th|10 th|tenth|x)(?:\s+(?:standard|grade|class|marks|score|percentage))?\s*(?::|is|are|=|-)?\s*(\d{1,3}(?:\.\d{1,2})?)',
            'Marks_12th': r'(?:12th|12 th|twelfth|xii)(?:\s+(?:standard|grade|class|marks|score|percentage))?\s*(?::|is|are|=|-)?\s*(\d{1,3}(?:\.\d{1,2})?)',
            'JEE_Score': r'(?:jee|joint entrance)(?:\s+(?:score|marks|rank|result))?\s*(?::|is|are|=|-)?\s*(\d+)',
            'Budget': r'(?:budget|afford|cost|fees|spending)(?:\s+(?:is|of|around|approximately|approx|about))?\s*(?::|=|-)?\s*(?:rs\.?|inr)?(?:\s*)(\d+(?:\.\d+)?(?:\s*(?:lakh|lakhs|lac|lacs|l|k|cr|crore|crores))?)',
            'Preferred_Location': r'(?:prefer|want|like)(?:red)?(?:\s+to\s+study)?\s+(?:in|at|near)\s+([a-zA-Z ]+)',
            'Gender': r'(?:gender|sex|i am a)\s*(?:is|:|=|-)?\s*\b(male|female|other|boy|girl|man|woman)\b',
            'Target_Exam': r'(?:preparing for|taking|giving|target(?:ing)?|writing)(?:\s+the)?\s+(jee|neet|clat|cat|gate)',
            'State_Board': r'(?:studied|completed|finished|did)(?:\s+(?:from|under|in))?\s+(cbse|icse|state board|international)',
            'Category': r'(?:category|reservation|quota)\s*(?:is|:|=|-)?\s*\b(general|obc|sc|st)\b',
        }

        # Check for matches in the user message
        for key, pattern in patterns.items():
            match = re.search(pattern, message.lower())
            if match and not self.student_data[key]:
                self.student_data[key] = match.group(1)

        # Special handling for longer text fields
        # Extract Extra-Curriculars
        if not self.student_data['Extra_Curriculars'] and ('extracurricular' in message.lower() or 'extra curricular' in message.lower() or 'activities' in message.lower()):
            parts = re.split(r'(?:my extracurriculars are|my extra curriculars are|my activities include|i participate in|i am involved in|i do)', message.lower(), flags=re.IGNORECASE)
            if len(parts) > 1:
                self.student_data['Extra_Curriculars'] = parts[1].strip()

        # Extract Future Goals
        if not self.student_data['Future_Goal'] and ('goal' in message.lower() or 'aspiration' in message.lower() or 'future' in message.lower() or 'career' in message.lower()):
            parts = re.split(r'(?:my goal is|my aspiration is|in the future|my career goal|i want to become|i want to be)', message.lower(), flags=re.IGNORECASE)
            if len(parts) > 1:
                self.student_data['Future_Goal'] = parts[1].strip()

    def check_data_completion(self):
        """Check if all required data has been collected"""
        missing_fields = [k for k, v in self.student_data.items() if v is None]
        return len(missing_fields) == 0

    def get_missing_fields(self):
        """Get list of missing fields"""
        return [k for k, v in self.student_data.items() if v is None]

    def format_colleges_for_prompt(self):
        """Format college data for inclusion in the prompt"""
        colleges_text = "College Database:\n"
        for college in self.colleges:
            colleges_text += f"- {college['name']}: Location: {college['location']}, "
            colleges_text += f"Min JEE Rank (if applicable): {college['min_jee']}, "
            colleges_text += f"Approximate Fees: {college['fees']}, "
            colleges_text += f"Acceptance Rate: {college['acceptance_rate']}, "
            colleges_text += f"Specialties: {', '.join(college['specialties'])}\n"
        return colleges_text

    def chat(self, message, history):
        """Process user message and generate response"""
        # Extract information from the user message
        self.extract_information(message)

        # Prepare user message for LLM
        self.conversation_history.append({"role": "user", "content": message})

        # Check if we have all the required information
        if not self.data_collected:
            self.data_collected = self.check_data_completion()

        # Prepare custom instructions for LLM based on conversation state
        if self.data_collected and not any(msg['content'].startswith('PROVIDE RECOMMENDATIONS') for msg in self.conversation_history if msg['role'] == 'system'):
            # All data collected, time for recommendations
            student_profile = "\n".join([f"{k}: {v}" for k, v in self.student_data.items()])
            colleges_data = self.format_colleges_for_prompt()

            self.conversation_history.append({
                "role": "system",
                "content": f"""
                PROVIDE RECOMMENDATIONS NOW. All required information has been collected.

                Student Profile:
                {student_profile}

                {colleges_data}

                Based on this student's profile, recommend 3-5 suitable colleges or universities from the database that match their profile.
                For each recommendation, explain:
                1. Why this college is a good fit
                2. Key programs relevant to their interests
                3. Admission requirements and competitiveness
                4. Estimated costs and financial considerations

                Also provide 2-3 practical next steps the student should take to prepare their applications.
                Format your response in a friendly, encouraging manner while being honest about admission chances.
                """
            })
        elif not self.data_collected:
            # Still collecting data
            missing_fields = self.get_missing_fields()
            next_field = missing_fields[0] if missing_fields else None

            if next_field:
                self.conversation_history.append({
                    "role": "system",
                    "content": f"""
                    You still need to collect more information from the student.

                    Information collected so far:
                    {json.dumps({k: v for k, v in self.student_data.items() if v is not None}, indent=2)}

                    Focus on getting information for: {next_field}

                    Ask in a conversational, friendly way. Don't list all missing fields at once.
                    Remember to stay conversational and acknowledge what the student says.
                    """
                })

        try:
            # Generate response using Groq API
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.conversation_history,
                temperature=0.7,
                max_tokens=1000,
            )

            assistant_response = response.choices[0].message.content
        except Exception as e:
            assistant_response = f"I'm sorry, there was an error generating a response: {str(e)}"
            print(f"Error: {str(e)}")

        # Add assistant response to history
        self.conversation_history.append({"role": "assistant", "content": assistant_response})

        return assistant_response


def create_chatbot_interface(api_key):
    """Create the Gradio interface for the chatbot"""
    counselor = CollegeCounselorChatbot(api_key=api_key, name="Lauren")

    with gr.Blocks(title="Lauren - AI College Counselor") as app:
        gr.Markdown("# 🎓 Lauren - Your AI College Counselor")
        gr.Markdown("""
        Welcome to your personal college counseling session with Lauren!

        Share information about your academic background, preferences, and goals through natural conversation.
        Lauren will guide you through the process and provide personalized college recommendations for your profile.

        Let's get started!
        """)

        chatbot = gr.Chatbot(
            height=500,
            show_copy_button=True,
            # Use a simple fallback for avatar image in case the URL isn't accessible
            avatar_images=(None, "https://i.imgur.com/mY2MHR3.png")
        )
        msg = gr.Textbox(
            placeholder="Hi Lauren! I need help finding the right college...",
            container=False,
            scale=7
        )
        with gr.Row():
            submit = gr.Button("Send", variant="primary", scale=1)
            clear = gr.Button("Clear Conversation", scale=1)

        # Set up the greeting message
        initial_greeting = """
        👋 Hi there! I'm Lauren, your personal AI college counselor.

        I'm here to help you find the best colleges based on your academic profile, preferences, and future goals.
        Let's have a conversation about your background and what you're looking for!

        Could you start by telling me about your 10th and 12th standard marks?
        """

        # Set up interactions
        def respond(message, chat_history):
            if not message:
                return chat_history

            response = counselor.chat(message, chat_history)
            chat_history.append((message, response))
            return "", chat_history

        # Set up clear function
        def clear_conversation():
            # Reset the counselor with the same API key
            nonlocal counselor
            counselor = CollegeCounselorChatbot(api_key=api_key, name="Lauren")
            return []

        # Set up event handlers
        msg.submit(respond, [msg, chatbot], [msg, chatbot])
        submit.click(respond, [msg, chatbot], [msg, chatbot])
        clear.click(clear_conversation, None, chatbot)

        # Set initial message on page load
        chatbot.value = [("", initial_greeting)]

    return app


# Main application function
def main():
    # Get API key - provide options for different environments
    api_key = os.environ.get("GROQ_API_KEY")

    # If running in Google Colab, try to get from userdata
    if not api_key:
        try:
            from google.colab import userdata
            api_key = userdata.get('groq_api_key')
        except:
            pass

    # If still no API key, prompt the user
    if not api_key:
        print("No Groq API key found in environment variables.")
        api_key_input = gr.Textbox(
            label="Enter your Groq API Key",
            placeholder="Enter your Groq API key here",
            type="password"
        )

        def launch_app(key):
            if not key:
                return "Please enter a valid API key"

            app = create_chatbot_interface(key)
            app.launch(share=True)
            return "Chatbot launched!"

        with gr.Blocks() as setup_app:
            gr.Markdown("# Lauren College Counselor Setup")
            gr.Markdown("Please enter your Groq API key to start the chatbot")
            api_key_input.render()
            launch_button = gr.Button("Launch Chatbot")
            output = gr.Textbox(label="Status")
            launch_button.click(launch_app, inputs=api_key_input, outputs=output)

        setup_app.launch(share=True)
    else:
        # API key is available, launch the chatbot directly
        app = create_chatbot_interface(api_key)
        app.launch(share=True)

if __name__ == "__main__":
    main()

<ipython-input-10-185ebe932d89>:224: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://06a8c332d60a2b72f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
